### 전략
- 일자를 기준으로 하여 저pbr, 저per 필터링
- 그중 per>0이고 pbr>0인 종목을 필터링
- 부채비율 변화율<0이고 eps 변화율>0인 종목을 필터링 하였습니다.
- 마지막으로 매수를 위해 이격도가 평균 보다 낮은 값을 필터링하여 9개 종목을 스크리닝 합니다.

In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

from pykrx import stock

In [2]:
#업데이트의 편의를 위해 api 사용
data = stock.get_market_ohlcv("20240318", market="KOSDAQ")
data = data[['등락률', '종가']]
data.head()

,등락률,종가
티커,,
060310,14.04,3290
054620,-0.27,7290
265520,0.00,20800
211270,-1.30,15910
139050,-2.46,7540


### 1. 저per, 저pbr

In [3]:
#api 활용 데이터 - per, pbr
df = stock.get_market_fundamental_by_ticker(date='20240318', market="KOSDAQ")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,109.67,3.47,30,0.00,0
054620,10864,14.44,0.67,505,0.00,0
265520,17980,3.81,1.16,5463,1.30,270
211270,5997,55.44,2.65,287,0.44,70
139050,5010,0.00,1.50,0,0.00,0


In [4]:
df.describe() #per<5 고정 #PBR의 경우 상위 25% 값보다 작은 것

,BPS,PER,PBR,EPS,DIV,DPS
count,1698.000000,1698.000000,1698.000000,1698.000000,1698.000000,1698.000000
mean,6394.591873,36.191013,2.968916,572.537102,0.740406,86.898704
std,9783.363365,293.646768,13.948686,1411.139293,1.471688,270.090579
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1305.250000,0.000000,0.680000,0.000000,0.000000,0.000000
50%,3481.000000,5.970000,1.235000,86.000000,0.000000,0.000000
75%,7622.750000,20.320000,2.520000,564.750000,0.975000,70.000000
max,171357.000000,9442.860000,515.000000,21470.000000,14.410000,5100.000000


In [5]:
#업데이트 편의를 위해 넘파이 사용
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
126600,4.17,0.67
035760,0.00,0.45
023460,0.00,0.53
456440,0.00,0.00
290120,1.59,0.44
...,...,...
061250,0.00,0.62
192410,0.00,0.34
115160,0.00,0.44


In [6]:
df3 = pd.merge(data, df2, on='티커')
df3

,등락률,종가,PER,PBR
티커,,,,
126600,-0.68,4395,4.17,0.67
035760,1.83,77700,0.00,0.45
023460,0.22,1791,0.00,0.53
456440,-1.76,2230,0.00,0.00
290120,1.23,3290,1.59,0.44
...,...,...,...,...
061250,0.24,1641,0.00,0.62
192410,7.08,454,0.00,0.34
115160,-0.35,2835,0.00,0.44


### 2. PER > 0, PBR > 0

In [7]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]
df3.shape #per>0 이고 pbr>0인 65개의 항목을 추출합니다

(65, 3)

In [8]:
df3.index

Index(['126600', '290120', '050120', '151860', '052900', '040610', '016250',
       '024910', '121440', '035080', '091590', '039560', '032190', '017650',
       '007680', '005710', '140520', '067990', '088910', '013120', '038060',
       '012700', '241770', '101330', '053700', '023600', '037460', '017480',
       '038540', '093920', '011370', '035890', '011560', '024830', '115570',
       '036710', '037760', '352700', '115480', '127710', '054800', '013310',
       '306040', '003800', '073540', '054940', '009780', '073560', '032940',
       '030530', '012620', '377460', '024800', '119850', '024880', '033290',
       '052330', '124560', '104480', '081150', '106240', '225590', '003380',
       '023760', '045100'],
      dtype='object', name='티커')

### 3. 부채비율, eps 변화율

In [9]:
#부채비율을 가져오기 위해 requests 라이브러리를 활용하였습니다.
import pandas as pd
import json
import requests

ticker = 290120
url = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd=290120&fin_typ=4&freq_typ=Y&encparam=dHBSMjZyTzBlaDZGd2p2enFUeThsZz09&id=ZlEwemUxRm'
headers= {
'Referer':'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more',
'User-Agent' :
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
'X-Requested-With' :
'XMLHttpRequest' }

res = requests.get(url=url, headers=headers)
df = pd.read_html(res.text)[1]
df

주요재무정보                연간                                      \
        주요재무정보 2018/12  (IFRS연결) 2019/12  (IFRS연결) 2020/12  (IFRS연결)   
0          매출액               NaN           1575.00           1718.00   
1         영업이익               NaN            101.00            141.00   
2   영업이익(발표기준)               NaN            101.00            141.00   
3     세전계속사업이익               NaN            102.00             73.00   
4        당기순이익               NaN             80.00             58.00   
5    당기순이익(지배)               NaN             80.00             58.00   
6   당기순이익(비지배)               NaN              0.00              0.00   
7         자산총계               NaN           1283.00           1729.00   
8         부채총계               NaN            803.00           1229.00   
9         자본총계               NaN            480.00            501.00   
10    자본총계(지배)               NaN            467.00            500.00   
11   자본총계(비지배)               NaN             13.00              1.00   
12         자본금               NaN             54.00             54.00   
13    영업활동현금흐름               NaN            -48.00            210.00   
14    투자활동현금흐름               NaN           -355.00           -316.00   
15    재무활동현금흐름               NaN            332.00            233.00   
16       CAPEX               NaN             43.00             81.00   
17         FCF               NaN            -91.00            129.00   
18      이자발생부채               NaN            493.00            781.00   
19       영업이익률               NaN              6.41              8.21   
20        순이익률               NaN              5.07              3.36   
21      ROE(%)               NaN               NaN             11.93   
22      ROA(%)               NaN               NaN              3.83   
23        부채비율               NaN            167.21            245.41   
24       자본유보율               NaN            750.26            824.40   
25      EPS(원)               NaN            733.00            530.00   
26      PER(배)               NaN              9.73             14.10   
27      BPS(원)               NaN           4288.00           4593.00   
28      PBR(배)               NaN              1.66              1.63   
29    현금DPS(원)            200.00            200.00            200.00   
30     현금배당수익률              3.49              2.81              2.68   
31   현금배당성향(%)               NaN             27.28             37.76   
32  발행주식수(보통주)       10884000.00       10884000.00       10884299.00   

                                                             \
   2021/12  (IFRS연결) 2022/12  (IFRS연결) 2023/12(E)  (IFRS연결)   
0            1790.00           2260.00                  NaN   
1             114.00            223.00                  NaN   
2             114.00            223.00                  NaN   
3             118.00            303.00                  NaN   
4              82.00            239.00                  NaN   
5              83.00            238.00                  NaN   
6              -1.00              0.00                  NaN   
7            2182.00           2329.00                  NaN   
8            1526.00           1445.00                  NaN   
9             656.00            884.00                  NaN   
10            654.00            879.00                  NaN   
11              2.00              5.00                  NaN   
12             57.00             59.00                  NaN   
13             59.00            199.00                  NaN   
14           -118.00           -140.00                  NaN   
15            253.00            -44.00                  NaN   
16            137.00             96.00                  NaN   
17            -78.00            103.00                  NaN   
18           1051.00            985.00                  NaN   
19              6.36              9.88                  NaN   
20              4.61             10.56                  NaN   
21             14.39             31.10

In [10]:
import pandas as pd
import json
import requests

# 분석할 종목들
tickers = df3.index

# 결과를 저장할 데이터프레임
result_df = pd.DataFrame(columns=['종목', '변화율'])

# 각 종목에 대해 반복하여 데이터 수집 및 분석 수행
for ticker in tickers:
    # URL 및 Referer 생성
    url_template = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={}&fin_typ=4&freq_typ=Y&encparam=dHBSMjZyTzBlaDZGd2p2enFUeThsZz09&id=ZlEwemUxRm'
    url = url_template.format(ticker)
    referer_template = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more'
    referer = referer_template.replace('290120', ticker)
    
    # HTTP 요청
    headers = {
        'Referer': referer,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }
    res = requests.get(url=url, headers=headers)
    df = pd.read_html(res.text)[1]
    
    # 데이터 전처리 및 분석
    df = df[[('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'), ('연간', '2022/12  (IFRS연결)')]]
    df = df.iloc[23:28]
    df['변화율'] = (df.iloc[:, 2] - df.iloc[:, 1]) / df.iloc[:, 1]
    
    # 종목별 결과를 데이터프레임에 추가
    df['종목'] = ticker   # '종목' 열에 티커 값 할당
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,부채비율,96.48,74.38,-0.229063,126600
1,NaN,NaN,자본유보율,1109.69,1301.61,0.172949,126600
2,NaN,NaN,EPS(원),35.00,998.00,27.514286,126600
3,NaN,NaN,PER(배),295.33,4.58,-0.984492,126600
4,NaN,NaN,BPS(원),6059.00,6427.00,0.060736,126600
...,...,...,...,...,...,...,...
320,NaN,NaN,부채비율,51.36,54.87,0.068341,045100
321,NaN,NaN,자본유보율,4511.80,5197.59,0.151999,045100
322,NaN,NaN,EPS(원),2547.00,3876.00,0.521790,045100
323,NaN,NaN,PER(배),6.62,3.73,-0.436556,045100


In [11]:
result_df.columns

Index([                       '종목',                       '변화율',
              ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [12]:
grouped_df = result_df.groupby(('종목', ''))
grouped_df.head()

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,부채비율,96.48,74.38,-0.229063,126600
1,NaN,NaN,자본유보율,1109.69,1301.61,0.172949,126600
2,NaN,NaN,EPS(원),35.00,998.00,27.514286,126600
3,NaN,NaN,PER(배),295.33,4.58,-0.984492,126600
4,NaN,NaN,BPS(원),6059.00,6427.00,0.060736,126600
...,...,...,...,...,...,...,...
320,NaN,NaN,부채비율,51.36,54.87,0.068341,045100
321,NaN,NaN,자본유보율,4511.80,5197.59,0.151999,045100
322,NaN,NaN,EPS(원),2547.00,3876.00,0.521790,045100
323,NaN,NaN,PER(배),6.62,3.73,-0.436556,045100


In [13]:
df = result_df.drop(columns=['종목', '변화율'])
df.columns

Index([       ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [14]:
df = df.rename(columns={('주요재무정보', '주요재무정보'): '재무비율', 
                        ('연간', '2021/12  (IFRS연결)'): '2021/12', 
                        ('연간', '2022/12  (IFRS연결)'): '2022/12', 
                        ('변화율', ''): '변화율', 
                        ('종목', ''): '종목'})

In [15]:
df.columns

Index(['재무비율', '2021/12', '2022/12', '변화율', '종목'], dtype='object')

In [16]:
df

,재무비율,2021/12,2022/12,변화율,종목
0,부채비율,96.48,74.38,-0.229063,126600
1,자본유보율,1109.69,1301.61,0.172949,126600
2,EPS(원),35.00,998.00,27.514286,126600
3,PER(배),295.33,4.58,-0.984492,126600
4,BPS(원),6059.00,6427.00,0.060736,126600
...,...,...,...,...,...
320,부채비율,51.36,54.87,0.068341,045100
321,자본유보율,4511.80,5197.59,0.151999,045100
322,EPS(원),2547.00,3876.00,0.521790,045100
323,PER(배),6.62,3.73,-0.436556,045100


In [17]:
#부채비율이 감소한 기업 추가 필터링
df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0) & (df['2022/12'] < 200)]['종목'].unique()
temp1 = df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0)]
temp1.shape 

(35, 5)

In [18]:
#EPS가 증가한 기업 필터링 
df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]['종목'].unique()
temp2 = df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]
temp2.shape 

(31, 5)

In [19]:
#부채비율이 감소하였으면서, eps가 증가한 기업 필터링
final = pd.merge(temp1, temp2, on='종목')
final['종목'] #재무비율 활용 끝

0     126600
1     290120
2     050120
3     151860
4     052900
5     040610
6     024910
7     091590
8     032190
9     005710
10    140520
11    101330
12    053700
13    023600
14    115570
15    009780
16    024800
17    124560
18    106240
Name: 종목, dtype: object

In [20]:
data = stock.get_market_ohlcv('20240215', '20240315', ticker)
data = data[['종가']]
std_dev = data['종가'].rolling(20).std()

data['중심 밴드'] = data['종가'].rolling(20).mean()
data['이격도'] = (data['종가'] / data['중심 밴드']) * 100
data.dropna()

,종가,중심 밴드,이격도
날짜,,,
2024-03-14,17150,16567.5,103.515920
2024-03-15,16770,16526.5,101.473391


In [21]:
ticker_list = final['종목'].tolist()
ticker_list

['126600',
 '290120',
 '050120',
 '151860',
 '052900',
 '040610',
 '024910',
 '091590',
 '032190',
 '005710',
 '140520',
 '101330',
 '053700',
 '023600',
 '115570',
 '009780',
 '024800',
 '124560',
 '106240']

### 4. 매수 위해 이격도 확인

In [53]:
#이격도가 이동평균선보다 낮은 티커 필터링
result_df = pd.DataFrame()
tickers = ticker_list

for ticker in tickers : 
    data = stock.get_market_ohlcv('20240215', '20240318', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std() 
    
    data['중심 밴드'] = data['종가'].rolling(20).mean() 
    data['상단'] = data['중심 밴드'] + 2 * data['종가'].rolling(window=20).std()
    data['하단'] = data['중심 밴드'] - 2 * data['종가'].rolling(window=20).std()
    data['이격도'] = (data['종가'] / data['중심 밴드']) * 100 
    data = data.dropna()
    
    result_df[ticker] = data['이격도']

result_df = result_df.T

In [54]:
data

,종가,중심 밴드,상단,하단,이격도
날짜,,,,,
2024-03-14,1464,1546.55,1709.488122,1383.611878,94.662313
2024-03-15,1483,1540.30,1702.902648,1377.697352,96.279945
2024-03-18,1500,1533.65,1691.086403,1376.213597,97.805888


In [60]:
import plotly.graph_objs as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['종가'], mode='lines', name='종가'))
fig.add_trace(go.Scatter(x=data.index, y=data['중심 밴드'], mode='lines', name='중심밴드'))


fig.add_trace(go.Scatter(x=data.index, y=data['상단'], mode='lines', line=dict(color='red'), name='상단'))
fig.add_trace(go.Scatter(x=data.index, y=data['하단'], mode='lines', line=dict(color='blue'), name='하단'))
fig.update_layout(title='볼린저 밴드', xaxis_title='날짜', yaxis_title='가격', template='plotly_dark')

fig.show()

In [24]:
result_df.describe()

날짜,2024-03-14,2024-03-15,2024-03-18
count,19.000000,19.000000,19.000000
mean,96.058703,96.556314,96.658223
std,2.496351,2.651914,2.376016
min,91.470509,92.055485,92.475068
25%,94.371464,94.548776,94.764466
50%,96.398816,96.864226,96.715630
75%,97.873125,98.549001,98.815068
max,100.457163,100.941128,101.030499


In [25]:
result_df.columns

DatetimeIndex(['2024-03-14', '2024-03-15', '2024-03-18'], dtype='datetime64[ns]', name='날짜', freq=None)

In [26]:
#추후 수정 필요
#넘파이 사용하려니 너무 길어져서 수작업으로 수정 필요
final_df = result_df.loc[(result_df.iloc[:, 0] < 97) & (result_df.iloc[:, 1] < 97) & (result_df.iloc[:, 0] < 97)]
final_df.shape

(9, 3)

In [27]:
final_df.index

Index(['290120', '050120', '091590', '053700', '023600', '115570', '024800',
       '124560', '106240'],
      dtype='object')

In [28]:
final_df.info

<bound method DataFrame.info of 날짜      2024-03-14  2024-03-15  2024-03-18
290120   92.774057   92.421442   94.445242
050120   92.771619   94.023194   94.639490
091590   95.058984   95.074357   93.887784
053700   94.080616   93.248722   93.308451
023600   95.284438   96.184561   94.941218
115570   91.470509   92.055485   92.475068
024800   95.354796   96.727354   96.615109
124560   93.747736   92.657343   94.889442
106240   94.662313   96.279945   97.805888>

In [29]:
graph = pd.merge(df, final_df, left_on='종목', right_index=True)
graph = graph[['재무비율', '2021/12', '2022/12', '변화율', '종목']]
graph2 = graph.loc[graph['재무비율'] == 'PER(배)', ['변화율', '종목']]

import plotly.express as px

# 그래프 생성
fig = px.bar(graph2, x='종목', y='변화율', color='변화율', color_continuous_scale='blues')

# 레이아웃 설정
fig.update_layout(
    title='<b>종목별 PER 변화율<b>',
    xaxis_title='종목',
    yaxis_title='변화율',
    yaxis_tickformat="%",
    plot_bgcolor='rgba(0,0,0,0)',
    template='seaborn'
)

# 그래프 표시
fig.show()